In [1]:
import numpy as np
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import FK5, SkyCoord
from astropy.cosmology import Planck18 as cosmo
from astropy import units as u
from tqdm import tqdm

In [2]:
data = Table.read('SDSS_LAB2024_photometry.fits')

When computing the local environment it is common to define: A Primary galaxy (the one you are computing the local density for), the Neighbours sample (all the other galaxies that contribute to the environment metrics)

In [3]:
Nneigh = np.zeros_like(data['ra'])
AP_radius = 500 #kpc
DV_cut    = 1000 #km/s

NEIcoords = SkyCoord(ra=data['ra']*u.deg, dec=data['dec']*u.deg)

for gal in tqdm(range(len(data['id']))):

    PRIcoords = SkyCoord(ra=data['ra'][gal]*u.deg, dec=data['dec'][gal]*u.deg)
    PRIredshift = data['redshift'][gal]
    
    #Obtain a separation of all galaxies from the primary and convert to arcsec
    Separation = NEIcoords.separation(PRIcoords).to(u.arcsec)

    #Kpc to arcsec and obtain a Separation in Kpc
    arcsec_per_kpc = cosmo.arcsec_per_kpc_proper(PRIredshift)
    Separation_kpc = Separation.value/arcsec_per_kpc.value

    #Evaluate separation in redshift space
    deltav = np.abs(3e5*(data['redshift']-PRIredshift)/(1+PRIredshift))

    #Define the selection conditions
    condition =  (Separation_kpc>0) & (Separation_kpc<AP_radius) & (deltav<DV_cut)
    
    #Save the number of galaxies satisfying the selections. Please note these values do not include the primary galaxy.
    Nneigh[gal] = condition.sum()


  0%|          | 0/92483 [00:00<?, ?it/s]

 18%|█▊        | 16784/92483 [06:39<29:58, 42.08it/s]

In [ ]:
#Turn the number of galaxies into a surface density of galaxies.
Dens_05 = Nneigh/(np.pi*0.5**2)

In [ ]:
tabout = Table((data['id'], Dens_05), names=('id', 'dens_05'))
tabout.write('SDSS_env.fits', overwrite=True)